# Trying out PLaMo-13b
PLaMo-13Bは大規模な言語モデルで、日英2言語に対する高い性能を示しています。約130億個のパラメータを持ち、日本語と英語のベンチマークタスクで優れた結果を示しています。

公開データセットのみで学習され、Apache License 2.0の下でオープンソースで公開・提供されています。また、Books3データセットを使用せずに学習されており、様々な用途に利用可能であることも特徴としています​。

* Huggingface: https://huggingface.co/pfnet/plamo-13b
* 論文:
* blog: https://tech.preferred.jp/ja/blog/llm-plamo/

In [1]:
!pip install transformers accelerate safetensors sentencepiece bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.1 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained("pfnet/plamo-13b", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "pfnet/plamo-13b",
    trust_remote_code=True,
    device_map='auto',
    quantization_config=quantization_config
).eval()

A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-13b:
- tokenization_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-13b:
- modeling_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Model Parameter Count: {model.num_parameters():,.0f}")

Vocab size: 50432
Model Parameter Count: 6,808,089,600


In [5]:
%%time

text = "これからの人工知能技術は"
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
generated_tokens = model.generate(
    inputs=input_ids,
    max_new_tokens=32,
    do_sample=True,
    top_p=0.95,
    temperature=1.0,
)[0]
generated_text = tokenizer.decode(generated_tokens)
print(generated_text)

これからの人工知能技術は、人間の意識や価値観に影響を与えるか、あるいはそもそも人工知能自身が人間性をもっているのか、といった議論を交わす機会が多くなりそうです。この
CPU times: user 13 s, sys: 3.83 s, total: 16.8 s
Wall time: 19.5 s


## 試してみる

ベースモデルなので、テキストを与えてその続きとしてどんな生成をするのかを見てみます。

In [7]:
generation_config = dict(
    max_new_tokens=50,
    repetition_penalty=1.1,
    top_p=0.95,
    top_k=50,
    temperature=0.7,
    do_sample=True,
)

In [8]:
%%time

inputs = tokenizer("吾輩は", return_tensors="pt", return_attention_mask=False)

outputs = model.generate(
    **inputs.to(model.device),
    **generation_config
)
text = tokenizer.batch_decode(outputs)[0]
print(text)

吾輩は猫である』の文庫本を読んでいる。
「猫」は人間より賢い動物である。 猫の仕草が人間の心を惑わす。 猫には、「吾輩は猫である。名前はまだない」という
CPU times: user 18.5 s, sys: 5.86 s, total: 24.4 s
Wall time: 24.3 s


In [9]:
%%time

inputs = tokenizer("最新の自然言語処理では", return_tensors="pt", return_attention_mask=False)

outputs = model.generate(
    **inputs.to(model.device),
    **generation_config
)
text = tokenizer.batch_decode(outputs)[0]
print(text)

最新の自然言語処理では、自然言語処理技術を、機械学習や深層学習などの様々な分野に応用しています。
また、2016年は、ディープラーニングのブームが到来しました。そのブーム以降、Deep Learning(
CPU times: user 18.5 s, sys: 6.04 s, total: 24.5 s
Wall time: 24.5 s


In [10]:
%%time

inputs = tokenizer("例題: りんごが２つとみかんが３つあります。フルーツは合計で何個ありますか？\n回答: ", return_tensors="pt", return_attention_mask=False)

outputs = model.generate(
    **inputs.to(model.device),
    **generation_config
)
text = tokenizer.batch_decode(outputs)[0]
print(text)

例題: りんごが2つとみかんが3つあります。フルーツは合計で何個ありますか?
回答: 5個
解説: (1)まずはりんごの個数を数えましょう。
2個なので、次の2つの答えが出てきますね。
2×2=4個です。
(2)みかんは3
CPU times: user 20 s, sys: 6.91 s, total: 26.9 s
Wall time: 26.8 s
